In [ ]:
import sys; sys.path.append("..")
import IPython
if (ipy := IPython.get_ipython()) is not None:
    ipy.run_line_magic("load_ext", "autoreload")
    ipy.run_line_magic("autoreload", "2")
import numpy as np
from scipy.spatial.transform import Rotation as R

import ElasticRods
from elastic_rods import PeriodicRod, RodMaterial

from tencers import *

from py_newton_optimizer import NewtonOptimizerOptions

from Tencers.viewers import HybridViewer
from Tencers.springs import *

This notebooks creates a custom tencer (by providing a set of rods and springs) and computes its equilibrium state.

# Create rods

In [ ]:
# Create rod points

n_divisions = 103

p = 2
q = 3

t = np.linspace(0,2*np.pi,n_divisions,endpoint=True)
t = np.concatenate([t, [t[1]]])  # last and first edge overlap

r = np.cos(q*t) + 2.5

x = r * np.cos(p*t) * 10.8
z = r * np.sin(p*t) * 10.8
y = -np.sin(q*t) * 10.8

rod_points = np.column_stack([x, y, z])

# Create rod

rod = PeriodicRod(rod_points, zeroRestCurvature=True)

rod_youngs_modulus = 1e6
material = RodMaterial('ellipse', rod_youngs_modulus, 0.5, [0.1, 0.1])  # circular cross-section
rod.setMaterial(material)

In [ ]:
# Create a second rod
rod_points_rotated = R.from_euler('z', np.pi).apply(rod_points)
rod2 = PeriodicRod(rod_points_rotated, zeroRestCurvature=True)
rod2.setMaterial(material)

In [ ]:
rods = [rod,rod2]
target_rods = [rod.rod, rod2.rod]

# Create springs

In [ ]:
# Spring endpoints on rods 
# Attachment vertex = [rod_idx_1, vertex_1, rod_idx_2, vertex_2]
attachment_vertices = [[0,17,1,85],[0,51,1,51],[0,85,1,17],
                       [0,30,1,47],[0,64,1,13],[0,98,1,81],[0,47,1,30],[0,13,1,64],[0,81,1,98],
                       [0,5,1,22],[0,39,1,90],[0,73,1,56],[0,22,1,5],[0,90,1,39],[0,56,1,73]]

# Spring stiffnesses
stiffnesses = [2.15700317e-02, 2.15700317e-02, 2.15700317e-02, 1.53859013e-02,
               1.53859013e-02, 1.53859013e-02, 1.53859013e-02, 1.53859013e-02,
               1.53859013e-02, 1.96186324e-02, 1.96186324e-02, 1.96186324e-02,
               1.96186324e-02, 1.96186324e-02, 1.96186324e-02]

# Spring rest lengths
rest_lengths = np.zeros(len(attachment_vertices))

# Create springs
springs,attachment_vertices_ = create_multi_rod_springs(rods,attachment_vertices,stiffness=stiffnesses,rest_lengths=rest_lengths)

# Create tencer
tencer = Tencer([],rods,springs,attachment_vertices_,targets=target_rods)

# Compute equilibrium state

In [ ]:
# Equilibrium options
opt = NewtonOptimizerOptions()
opt.useNegativeCurvatureDirection = True
opt.niter = 500
opt.hessianScaledBeta = False
fixed_vars = []

# Compute equilibrium
c = computeEquilibrium(tencer,fixedVars=fixed_vars, opts=opt, hessianShift=1e-8)

In [ ]:
# View result
viewer = HybridViewer([tencer], wireFrame = True)
viewer.show()